# VGG16 model

In [38]:
import cv2
import keras
from pathlib import Path
import tensorflow as tf
import joblib
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np


def __preprocess_image(path):
    img = cv2.imread(path)
    if img is not None:
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      img = cv2.resize(img, (224,224))  # width, height
      img = img.astype("float32") / 255.0
      #img = np.reshape(img, self.image_size)  # (height, width, channels)
      return img
    else:
      print("Image not found")
      return None


def mlb_load(file_path:Path)->MultiLabelBinarizer:

    try:
        # Assuming you run this notebook from the root of your project directory
        mlb = joblib.load(file_path)

    except FileNotFoundError:
        print("Error: 'artifacts/model/vgg_model/mlb.joblib' not found.")
        print("Please make sure the path is correct. Using a placeholder binarizer.")
        # As a placeholder, let's create a dummy mlb if the file is not found.
        mlb = MultiLabelBinarizer()
        # This should be the set of your actual labels.
        mlb.fit([['advertisement', 'email', 'form', 'invoice', 'note']])

    return mlb


def load_vgg_model() -> tuple[tf.keras.Model, MultiLabelBinarizer]:
  try:

      mlb_file_path=Path("/content/drive/MyDrive/Work_space/Project/document_classification/artifacts/model/vgg_model/mlb.joblib")
      model_file_path=Path("/content/drive/MyDrive/Work_space/Project/document_classification/artifacts/model/vgg_model/model.keras")
      # Select model
      # Load the entire model
      mlb=mlb_load(mlb_file_path)
      model=tf.keras.models.load_model(model_file_path) # Corrected to use tf.keras.models.load_model
      return model,mlb

  except Exception as e:
      raise e


try:
  model,mlb=load_vgg_model()
except Exception as e:
  print(e)

def vgg_model_prediction(image_path):
  # Check if image is valid
  image=__preprocess_image(image_path)
  if image is not None:
      # Add batch dimension to the image
      image = np.expand_dims(image, axis=0)
      prd = model.predict(image)
      # Convert the prediction to a binary indicator format
      pred_id = np.argmax(prd, axis=1)
      #print(pred_id)
      # Create a zero array with the shape (1, number of classes)
      binary_prediction = np.zeros((1, len(mlb.classes_)))
      #print(binary_prediction)
      # Set the index of the predicted class to 1
      binary_prediction[0, pred_id] = 1
      #print(binary_prediction)
      predicted_labels = mlb.inverse_transform(binary_prediction)
      #print(f"Predicted labels: {predicted_labels}")
      return {"status":1,"message":predicted_labels[0][0]}
  else:
      return {"status":0, "message": "Image not processed."}

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MultiLabelBinarizer from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [40]:
vgg_model_prediction("/content/38.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 594ms/step


{'status': 1, 'message': 'invoice'}

## Production grade

In [62]:
import logging
import joblib
import tensorflow as tf
from pathlib import Path
from sklearn.preprocessing import MultiLabelBinarizer

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_vgg_artifacts(model_path: Path, mlb_path: Path) -> tuple[tf.keras.Model, MultiLabelBinarizer]:
    """
    Loads the VGG model and the MultiLabelBinarizer from specified paths.

    Args:
        model_path: Path to the VGG model file (.keras).
        mlb_path: Path to the MultiLabelBinarizer file (.joblib).

    Returns:
        A tuple containing the loaded Keras model and MultiLabelBinarizer object.

    Raises:
        FileNotFoundError: If either the model file or the MLB file is not found.
        Exception: If any other error occurs during loading.
    """
    model = None
    mlb = None
    try:
        logging.info(f"Attempting to load VGG model from {model_path}")
        model = tf.keras.models.load_model(model_path)
        logging.info("VGG model loaded successfully.")
    except FileNotFoundError:
        logging.error(f"Error: VGG model file not found at {model_path}")
        raise
    except Exception as e:
        logging.error(f"An error occurred while loading the VGG model: {e}")
        raise

    try:
        logging.info(f"Attempting to load MultiLabelBinarizer from {mlb_path}")
        mlb = joblib.load(mlb_path)
        logging.info("MultiLabelBinarizer loaded successfully.")
    except FileNotFoundError:
        logging.error(f"Error: MultiLabelBinarizer file not found at {mlb_path}")
        raise
    except Exception as e:
        logging.error(f"An error occurred while loading the MultiLabelBinarizer: {e}")
        raise

    logging.info("Both VGG model and MultiLabelBinarizer loaded successfully.")
    return model, mlb



In [63]:
import cv2
import numpy as np
import logging
from pathlib import Path

# Configure logging (if not already configured in a previous cell)
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


def preprocess_image(image_path: Path, target_size: tuple[int, int] = (224, 224)) -> np.ndarray | None:
    """
    Preprocesses an image for VGG model prediction.

    Loads an image from the specified path, converts it to RGB, resizes it,
    and normalizes pixel values. Includes robust error handling and logging
    at each step.

    Args:
        image_path: Path to the image file.
        target_size: A tuple (width, height) specifying the desired output size.

    Returns:
        A preprocessed NumPy array representing the image with pixel values
        scaled between 0 and 1, or None if an error occurred during processing.
    """
    try:
        logging.info(f"Attempting to load image from {image_path}")
        img = cv2.imread(str(image_path)) # cv2.imread expects a string or numpy array

        if img is None:
            logging.error(f"Error: Could not load image from {image_path}. cv2.imread returned None.")
            return None
        logging.info("Image loaded successfully.")

        logging.info("Attempting to convert image to RGB.")
        # Check if the image is already in a format that doesn't need BGR to RGB conversion
        # cv2.imread loads in BGR format by default for color images.
        # If the image is grayscale, it might be loaded as such.
        # We want RGB for consistency with models trained on RGB data.
        if len(img.shape) == 3 and img.shape[2] == 3: # Check if it's a color image (likely BGR)
            try:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                logging.info("Image converted to RGB successfully.")
            except cv2.error as e:
                logging.error(f"Error during BGR to RGB conversion for image {image_path}: {e}")
                return None
        elif len(img.shape) == 2: # Grayscale image
             try:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                logging.info("Grayscale image converted to RGB successfully.")
             except cv2.error as e:
                logging.error(f"Error during Grayscale to RGB conversion for image {image_path}: {e}")
                return None
        else:
             logging.warning(f"Unexpected image format for {image_path}. Attempting to proceed.")
             # If it's not a standard color or grayscale, we might proceed but log a warning.
             # Depending on requirements, you might want to return None here.


        logging.info(f"Attempting to resize image to {target_size}.")
        try:
            img = cv2.resize(img, target_size)
            if img is None or img.size == 0:
                 logging.error(f"Error: cv2.resize returned None or empty array for image {image_path}.")
                 return None
            logging.info("Image resized successfully.")
        except cv2.error as e:
            logging.error(f"Error during image resizing for image {image_path} to size {target_size}: {e}")
            return None


        logging.info("Attempting to normalize pixel values.")
        try:
            # Ensure the image is the correct dtype before division
            img = img.astype("float32") / 255.0
            if img is None or img.size == 0 or np.max(img) > 1.0 or np.min(img) < 0.0:
                 logging.error(f"Error: Image normalization failed or resulted in unexpected values for image {image_path}.")
                 return None
            logging.info("Pixel values normalized successfully.")
        except Exception as e:
            logging.error(f"Error during pixel normalization for image {image_path}: {e}")
            return None

        logging.info(f"Image preprocessing completed successfully for {image_path}.")
        return img

    except Exception as e:
        logging.error(f"An unexpected error occurred during image preprocessing for {image_path}: {e}")
        return None


In [64]:
import cv2
import keras
from pathlib import Path
import tensorflow as tf
import joblib
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import logging
from typing import Optional, Tuple, List

# Configure logging (if not already configured)
if not logging.getLogger('').handlers:
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info("Logging configured with basicConfig.")
else:
    logging.info("Logging already configured.")

class VGGDocumentClassifier:
    """
    A class for classifying documents using a VGG16 model.

    This class encapsulates the loading of the VGG16 model and its associated
    MultiLabelBinarizer, provides a method to preprocess input images, and
    performs document classification predictions.
    """

    def __init__(self, model_path: Path, mlb_path: Path, target_size: Tuple[int, int] = (224, 224)) -> None:
        """
        Initializes the VGGDocumentClassifier by loading the model and MLB.

        Args:
            model_path: Path to the VGG model file (.keras).
            mlb_path: Path to the MultiLabelBinarizer file (.joblib).
            target_size: The target size (width, height) for image preprocessing.
                         Defaults to (224, 224).

        Raises:
            FileNotFoundError: If either the model file or the MLB file is not found.
            Exception: If any other error occurs during loading.
        """
        logging.info("Initializing VGGDocumentClassifier.")
        self.model: Optional[tf.keras.Model] = None
        self.mlb: Optional[MultiLabelBinarizer] = None
        self.target_size: Tuple[int, int] = target_size

        try:
            self._load_artifacts(model_path, mlb_path)
            if self.model and self.mlb:
                logging.info("VGGDocumentClassifier initialized successfully.")
            else:
                logging.critical("VGGDocumentClassifier failed to fully initialize due to artifact loading errors.")
                raise RuntimeError("Failed to load all required artifacts for VGGDocumentClassifier.")
        except Exception as e:
            logging.critical(f"Failed to initialize VGGDocumentClassifier: {e}", exc_info=True)
            raise # Re-raise the exception after logging


    def _load_artifacts(self, model_path: Path, mlb_path: Path) -> None:
        """
        Loads the VGG model and MultiLabelBinarizer with error handling and logging.

        Args:
            model_path: Path to the VGG model file (.keras).
            mlb_path: Path to the MultiLabelBinarizer file (.joblib).

        Raises:
            FileNotFoundError: If either the model file or the MLB file is not found.
            Exception: If any other unexpected error occurs during loading.
        """
        logging.info("Starting artifact loading.")
        model_loaded: bool = False
        mlb_loaded: bool = False

        # Load Model
        try:
            logging.info(f"Attempting to load VGG model from {model_path}")
            self.model = tf.keras.models.load_model(model_path)
            logging.info("VGG model loaded successfully.")
            model_loaded = True
        except FileNotFoundError:
            logging.critical(f"Critical Error: VGG model file not found at {model_path}", exc_info=True)
            raise # Re-raise to indicate a critical initialization failure
        except Exception as e:
            logging.critical(f"Critical Error: An unexpected error occurred while loading the VGG model from {model_path}: {e}", exc_info=True)
            raise # Re-raise to indicate a critical initialization failure

        # Load MLB
        try:
            logging.info(f"Attempting to load MultiLabelBinarizer from {mlb_path}")
            self.mlb = joblib.load(mlb_path)
            logging.info("MultiLabelBinarizer loaded successfully.")
            mlb_loaded = True
        except FileNotFoundError:
            logging.critical(f"Critical Error: MultiLabelBinarizer file not found at {mlb_path}", exc_info=True)
            raise # Re-raise to indicate a critical initialization failure
        except Exception as e:
            logging.critical(f"Critical Error: An unexpected error occurred while loading the MultiLabelBinarizer from {mlb_path}: {e}", exc_info=True)
            raise # Re-raise to indicate a critical initialization failure

        if model_loaded and mlb_loaded:
             logging.info("All required VGG artifacts loaded successfully.")
        else:
            logging.error("One or more required VGG artifacts failed to load during _load_artifacts.")


    def preprocess_image(self, image_path: Path) -> Optional[np.ndarray]:
        """
        Preprocesses an image for VGG model prediction.

        Loads an image from the specified path, converts it to RGB, resizes it,
        and normalizes pixel values. Includes robust error handling and logging
        at each step.

        Args:
            image_path: Path to the image file.

        Returns:
            A preprocessed NumPy array representing the image with pixel values
            scaled between 0 and 1, or None if an error occurred during processing.
        """
        try:
            logging.info(f"Attempting to load image from {image_path}")
            img = cv2.imread(str(image_path)) # cv2.imread expects a string or numpy array

            if img is None:
                logging.error(f"Error: Could not load image from {image_path}. cv2.imread returned None.")
                return None
            logging.info("Image loaded successfully.")

            logging.info("Attempting to convert image to RGB.")
            if len(img.shape) == 3 and img.shape[2] == 3: # Check if it's a color image (likely BGR)
                try:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    logging.info("Image converted to RGB successfully.")
                except cv2.error as e:
                    logging.error(f"Error during BGR to RGB conversion for image {image_path}: {e}")
                    return None
            elif len(img.shape) == 2: # Grayscale image
                 try:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                    logging.info("Grayscale image converted to RGB successfully.")
                 except cv2.error as e:
                    logging.error(f"Error during Grayscale to RGB conversion for image {image_path}: {e}")
                    return None
            else:
                 logging.warning(f"Unexpected image format for {image_path}. Attempting to proceed.")


            logging.info(f"Attempting to resize image to {self.target_size}.")
            try:
                img = cv2.resize(img, self.target_size)
                if img is None or img.size == 0:
                     logging.error(f"Error: cv2.resize returned None or empty array for image {image_path}.")
                     return None
                logging.info("Image resized successfully.")
            except cv2.error as e:
                logging.error(f"Error during image resizing for image {image_path} to size {self.target_size}: {e}")
                return None


            logging.info("Attempting to normalize pixel values.")
            try:
                img = img.astype("float32") / 255.0
                if img is None or img.size == 0 or np.max(img) > 1.0 or np.min(img) < 0.0:
                     logging.error(f"Error: Image normalization failed or resulted in unexpected values for image {image_path}.")
                     return None
                logging.info("Pixel values normalized successfully.")
            except Exception as e:
                logging.error(f"Error during pixel normalization for image {image_path}: {e}")
                return None

            logging.info(f"Image preprocessing completed successfully for {image_path}.")
            return img

        except Exception as e:
            logging.error(f"An unexpected error occurred during image preprocessing for {image_path}: {e}")
            return None


    def predict(self, image_path: Path) -> Optional[List[str]]:
        """
        Predicts the class labels for a given image using the loaded VGG model.

        The process involves loading and preprocessing the image, performing
        inference with the model, and converting the prediction to class labels
        using the MultiLabelBinarizer.

        Args:
            image_path: Path to the image file to classify.

        Returns:
            A list of predicted class labels (strings) if the prediction process
            is successful. Returns None if any critical step (image loading,
            preprocessing, model inference, or inverse transform) fails.
            Returns an empty list if the prediction process is successful but
            no labels are predicted.
        """
        logging.info(f"Starting prediction process for image: {image_path}.")

        if self.model is None or self.mlb is None:
            logging.error("Model or MultiLabelBinarizer not loaded. Cannot perform prediction.")
            return None

        # Preprocess image
        image = self.preprocess_image(image_path)
        if image is None:
            logging.error(f"Image preprocessing failed for {image_path}. Cannot perform prediction.")
            return None

        try:
            logging.info(f"Performing model inference for {image_path}.")
            # Add batch dimension to the image
            image = np.expand_dims(image, axis=0)
            prd = self.model.predict(image)
            logging.info(f"Model inference completed for {image_path}. Prediction shape: {prd.shape}")
        except Exception as e:
            logging.error(f"An error occurred during model inference for {image_path}: {e}", exc_info=True)
            return None


        # Convert the prediction to a binary indicator format and get labels
        try:
            logging.info(f"Converting prediction to labels for {image_path}.")
            # Assuming multi-class classification for now, taking the argmax
            # If it's multi-label, you'd apply a sigmoid and thresholding here
            pred_id = np.argmax(prd, axis=1)

            # Create a zero array with the shape (1, number of classes)
            binary_prediction = np.zeros((1, len(self.mlb.classes_)))
            # Set the index of the predicted class to 1
            binary_prediction[0, pred_id] = 1


            predicted_labels_tuple_list: List[Tuple[str, ...]] = self.mlb.inverse_transform(binary_prediction)
            logging.info(f"Prediction processed for {image_path}. Predicted labels (raw tuple list): {predicted_labels_tuple_list}")

            if predicted_labels_tuple_list and len(predicted_labels_tuple_list) > 0:
                 final_labels: List[str] = list(predicted_labels_tuple_list[0])
                 logging.info(f"Final predicted labels for {image_path}: {final_labels}")
                 return final_labels
            else:
                 logging.warning(f"MLB inverse_transform returned an empty list for {image_path}. No labels predicted.")
                 return []

        except Exception as e:
            logging.error(f"An error occurred during inverse transform or label processing for {image_path}: {e}", exc_info=True)
            return None

2025-06-30 11:42:06,061 - INFO - Logging already configured.


In [66]:
model_path=Path("/content/drive/MyDrive/Work_space/Project/document_classification/artifacts/model/vgg_model/model.keras")
mlb_path=Path("/content/drive/MyDrive/Work_space/Project/document_classification/artifacts/model/vgg_model/mlb.joblib")

classifier = VGGDocumentClassifier(model_path,mlb_path)

predicted_labels = classifier.predict(Path("/content/38.jpg"))

2025-06-30 11:42:36,313 - INFO - Initializing VGGDocumentClassifier.
2025-06-30 11:42:36,325 - INFO - Starting artifact loading.
2025-06-30 11:42:36,328 - INFO - Attempting to load VGG model from /content/drive/MyDrive/Work_space/Project/document_classification/artifacts/model/vgg_model/model.keras
2025-06-30 11:42:37,908 - INFO - VGG model loaded successfully.
2025-06-30 11:42:37,909 - INFO - Attempting to load MultiLabelBinarizer from /content/drive/MyDrive/Work_space/Project/document_classification/artifacts/model/vgg_model/mlb.joblib
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MultiLabelBinarizer from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
2025-06-30 11:42:37,915 - INFO - MultiLabelBinarizer loade

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


2025-06-30 11:42:39,436 - INFO - Model inference completed for /content/38.jpg. Prediction shape: (1, 5)
2025-06-30 11:42:39,437 - INFO - Converting prediction to labels for /content/38.jpg.
2025-06-30 11:42:39,439 - INFO - Prediction processed for /content/38.jpg. Predicted labels (raw tuple list): [('invoice',)]
2025-06-30 11:42:39,440 - INFO - Final predicted labels for /content/38.jpg: ['invoice']


In [67]:
print(predicted_labels )

['invoice']


#VIT

In [ ]:
import joblib
from sklearn.preprocessing import MultiLabelBinarizer
from pathlib import Path
import torch
import numpy as np
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification



try:

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    mlb_file_path=Path("artifacts\model\VIT_model\mlb.joblib")
    model_file_path=Path("artifacts\model\VIT_model\model.pth")
    # Select model
    model_id = "google/vit-base-patch16-224-in21k"
    # Load processor
    processor = AutoImageProcessor.from_pretrained(model_id, use_fast=True)

    # TODO: You need to load your fine-tuned model here
    # For example:
    # model = AutoModelForImageClassification.from_pretrained("path/to/your/fine-tuned-model")
    # For now, we will use the base model for demonstration, but it will not give correct predictions.
    #model = AutoModelForImageClassification.from_pretrained(model_id)
    # Load the entire model
    model= torch.load(model_file_path, map_location=device,weights_only=False )
    # Set device
    model.to(device)

except Exception as e:
    raise e




def mlb_load(file_path:Path)->MultiLabelBinarizer:
    try:
        # Assuming you run this notebook from the root of your project directory
        mlb = joblib.load(file_path)

    except FileNotFoundError:
        print("Error: 'artifacts/model/VIT_model/mlb.joblib' not found.")
        print("Please make sure the path is correct. Using a placeholder binarizer.")
        # As a placeholder, let's create a dummy mlb if the file is not found.
        mlb = MultiLabelBinarizer()
        # This should be the set of your actual labels.
        mlb.fit([['advertisement', 'email', 'form', 'invoice', 'note']])
    return mlb






def VIT_model_prediction(image_path:Path,cut_off:float):
    try:
        # Load and convert image
        # --- IMPORTANT: Please update this path to your image ---
        try:
            image = Image.open(image_path)
            if image.mode != "RGB":
                image = image.convert("RGB")
        except FileNotFoundError:
            print(f"Error: Image not found at {image_path}")
            print("Using a dummy image for demonstration.")
            # Create a dummy image for demonstration if image not found
            image = Image.new('RGB', (224, 224), color = 'red')


        # Preprocess image
        pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)

        # Forward pass
        with torch.no_grad():
            outputs = model(pixel_values)
            logits = outputs.logits

        # Apply sigmoid for multi-label classification
        sigmoid = torch.nn.Sigmoid()
        probs = sigmoid(logits.squeeze().cpu())

        # Thresholding (using 0.5 as an example)
        predictions = np.zeros(probs.shape)
        predictions[np.where(probs >= cut_off)] = 1

        # Get label names using the loaded MultiLabelBinarizer
        mlb=mlb_load(mlb_file_path)
        # The predictions need to be in a 2D array for inverse_transform, e.g., (1, num_classes)
        predicted_labels = mlb.inverse_transform(predictions.reshape(1, -1))
        print(f"Predicted labels: {predicted_labels}")
        return {"status":1,}

    except Exception as e:
        raise e



VIT_model_prediction(Path("dataset\sample_text_ds\test\email\2078379610a.jpg"),0.5)

## Production grade

In [ ]:
import logging
import numpy as np
import torch
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification
from sklearn.preprocessing import MultiLabelBinarizer
import joblib
from pathlib import Path
from typing import List, Optional, Tuple, Any

# Configure logging (if not already configured, set a basic configuration)
# Check if handlers exist to avoid adding multiple handlers in interactive environments
if not logging.getLogger('').handlers:
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info("Logging configured with basicConfig.")
else:
    logging.info("Logging already configured.")


class VITDocumentClassifier:
    """
    A class for classifying documents using a Vision Transformer (ViT) model.

    This class encapsulates the loading of the ViT model, its associated processor,
    and a MultiLabelBinarizer for converting model outputs to meaningful labels.
    It provides a method to preprocess input images and perform multi-label
    classification predictions with a specified confidence cutoff threshold.
    """

    def __init__(self, model_path: Path, mlb_path: Path, model_id: str = "google/vit-base-patch16-224-in21k") -> None:
        """
        Initializes the VITDocumentClassifier by loading the model, processor, and MLB.

        Args:
            model_path: Path to the ViT model file (.pth). This is expected to be
                        a pre-trained or fine-tuned PyTorch model file.
            mlb_path: Path to the MultiLabelBinarizer file (.joblib). This file
                      should contain the fitted binarizer object corresponding
                      to the model's output classes.
            model_id: The Hugging Face model ID for the processor. This is used
                      to load the appropriate image processor for the ViT model.
                      Defaults to "google/vit-base-patch16-224-in21k".

        Raises:
            FileNotFoundError: If either the model file or the MLB file is not found
                             at the specified paths during artifact loading.
            Exception: If any other unexpected error occurs during the loading
                       of the model, processor, or MultiLabelBinarizer.
            RuntimeError: If artifact loading fails for critical components
                          (model or MLB).
        """
        logging.info("Initializing VITDocumentClassifier.")
        self.model: Optional[torch.nn.Module] = None
        self.processor: Optional[AutoImageProcessor] = None
        self.mlb: Optional[MultiLabelBinarizer] = None
        self.device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        logging.info(f"Using device: {self.device}")
        self.model_id: str = model_id

        try:
            self._load_artifacts(model_path, mlb_path)
            if self.model and self.processor and self.mlb:
                logging.info("VITDocumentClassifier initialized successfully.")
            else:
                # This case should ideally be caught and re-raised in _load_artifacts
                # but adding a check here for robustness.
                logging.critical("VITDocumentClassifier failed to fully initialize due to artifact loading errors.")
                raise RuntimeError("Failed to load all required artifacts for VITDocumentClassifier.")

        except Exception as e:
            logging.critical(f"Failed to initialize VITDocumentClassifier: {e}", exc_info=True)
            # Re-raise the exception after logging
            raise


    def _load_artifacts(self, model_path: Path, mlb_path: Path) -> None:
        """
        Loads the ViT model, processor, and MultiLabelBinarizer with enhanced error handling and logging.

        This is an internal helper method called during initialization.

        Args:
            model_path: Path to the ViT model file (.pth).
            mlb_path: Path to the MultiLabelBinarizer file (.joblib).

        Raises:
            FileNotFoundError: If either the model file or the MLB file is not found.
            Exception: If any other unexpected error occurs during loading.
        """
        logging.info("Starting artifact loading.")
        processor_loaded: bool = False
        model_loaded: bool = False
        mlb_loaded: bool = False

        # Load Processor
        try:
            logging.info(f"Attempting to load ViT processor for model ID: {self.model_id}")
            self.processor = AutoImageProcessor.from_pretrained(self.model_id, use_fast=True)
            logging.info("ViT processor loaded successfully.")
            processor_loaded = True
        except Exception as e:
            # Log at error level as processor is important but not strictly critical if we raise later
            logging.error(f"An error occurred while loading the ViT processor for model ID {self.model_id}: {e}", exc_info=True)
            # Do not re-raise here, continue loading other artifacts


        # Load Model
        try:
            logging.info(f"Attempting to load ViT model from {model_path}")
            # Note: Adjust map_location as needed based on where the model was saved
            self.model = torch.load(model_path, map_location=self.device, weights_only=False)
            self.model.to(self.device) # Ensure model is on the correct device
            logging.info(f"ViT model loaded successfully and moved to {self.device}.")
            model_loaded = True
        except FileNotFoundError:
            logging.critical(f"Critical Error: ViT model file not found at {model_path}", exc_info=True)
            raise # Re-raise to indicate a critical initialization failure
        except Exception as e:
            logging.critical(f"Critical Error: An unexpected error occurred while loading the ViT model from {model_path}: {e}", exc_info=True)
            raise # Re-raise to indicate a critical initialization failure


        # Load MLB
        try:
            logging.info(f"Attempting to load MultiLabelBinarizer from {mlb_path}")
            self.mlb = joblib.load(mlb_path)
            logging.info("MultiLabelBinarizer loaded successfully.")
            mlb_loaded = True
        except FileNotFoundError:
            logging.critical(f"Critical Error: MultiLabelBinarizer file not found at {mlb_path}", exc_info=True)
            raise # Re-raise to indicate a critical initialization failure
        except Exception as e:
            logging.critical(f"Critical Error: An unexpected error occurred while loading the MultiLabelBinarizer from {mlb_path}: {e}", exc_info=True)
            raise # Re-raise to indicate a critical initialization failure

        if processor_loaded and model_loaded and mlb_loaded:
             logging.info("All required ViT artifacts loaded successfully.")
        else:
            logging.error("One or more required ViT artifacts failed to load during _load_artifacts.")


    def predict(self, image_path: Path, cut_off: float = 0.5) -> Optional[List[str]]:
        """
        Predicts the class labels for a given image using the loaded ViT model.

        The process involves loading and preprocessing the image, performing
        inference with the model, applying a sigmoid activation, thresholding
        the probabilities to obtain binary predictions, and finally converting
        the binary predictions back to class labels using the MultiLabelBinarizer.

        Args:
            image_path: Path to the image file to classify. The image is expected
                        to be in a format compatible with PIL (Pillow).
            cut_off: The threshold for converting predicted probabilities into
                     binary labels. Probabilities greater than or equal to this
                     value are considered positive predictions (1), otherwise 0.
                     Defaults to 0.5.

        Returns:
            A list of predicted class labels (strings) if the prediction process
            is successful. Returns None if any critical step (image loading,
            preprocessing, model inference, or inverse transform) fails.
            Returns an empty list if the prediction process is successful but
            no labels meet the cutoff threshold.
        """
        logging.info(f"Starting prediction process for image: {image_path} with cutoff {cut_off}.")

        if self.model is None or self.processor is None or self.mlb is None:
            logging.error("Model, processor, or MultiLabelBinarizer not loaded. Cannot perform prediction.")
            return None

        # Load and preprocess image
        image: Optional[Image.Image] = None
        try:
            logging.info(f"Attempting to load image from {image_path}")
            image = Image.open(image_path)
            logging.info(f"Image loaded successfully from {image_path}.")
        except FileNotFoundError:
            logging.error(f"Error: Image file not found at {image_path}", exc_info=True)
            return None
        except Exception as e:
            logging.error(f"An unexpected error occurred while loading image {image_path}: {e}", exc_info=True)
            return None

        try:
            logging.info(f"Attempting to convert image to RGB for {image_path}.")
            if image.mode != "RGB":
                image = image.convert("RGB")
                logging.info(f"Image converted to RGB successfully for {image_path}.")
            else:
                 logging.info(f"Image is already in RGB format for {image_path}.")

        except Exception as e:
            logging.error(f"An error occurred while converting image {image_path} to RGB: {e}", exc_info=True)
            return None


        # Preprocess image using the loaded processor
        try:
            logging.info(f"Attempting to preprocess image using processor for {image_path}.")
            # Check if image is valid after loading/conversion
            if image is None:
                 logging.error(f"Image is None after loading/conversion for {image_path}. Cannot preprocess.")
                 return None
            # The processor expects a PIL Image or a list of PIL Images
            pixel_values: torch.Tensor = self.processor(images=image, return_tensors="pt").pixel_values.to(self.device)
            logging.info(f"Image preprocessed and moved to device ({self.device}).")
        except Exception as e:
            logging.error(f"An error occurred during image preprocessing for {image_path}: {e}", exc_info=True)
            return None

        # Forward pass
        try:
            logging.info(f"Starting model forward pass for {image_path}.")
            self.model.eval() # Set model to evaluation mode
            with torch.no_grad():
                outputs: Any = self.model(pixel_values) # Use Any because the output type can vary
                logits: torch.Tensor = outputs.logits
            logging.info(f"Model forward pass completed for {image_path}.")
        except Exception as e:
            logging.error(f"An error occurred during model forward pass for {image_path}: {e}", exc_info=True)
            return None


        # Apply sigmoid and thresholding
        try:
            logging.info(f"Applying sigmoid and thresholding for {image_path}.")
            sigmoid: torch.nn.Sigmoid = torch.nn.Sigmoid()
            probs: torch.Tensor = sigmoid(logits.squeeze().cpu())

            predictions: np.ndarray = np.zeros(probs.shape, dtype=int) # Explicitly set dtype to int
            predictions[np.where(probs >= cut_off)] = 1
            logging.info(f"Applied sigmoid and thresholding with cutoff {cut_off} for {image_path}. Binary predictions shape: {predictions.shape}")
        except Exception as e:
            logging.error(f"An error occurred during probability processing for {image_path}: {e}", exc_info=True)
            return None


        # Get label names using the loaded MultiLabelBinarizer
        try:
            logging.info(f"Performing inverse transform using MultiLabelBinarizer for {image_path}.")
            # The predictions need to be in a 2D array for inverse_transform, e.g., (1, num_classes)
            # Use the self.mlb loaded during initialization

            # Ensure self.mlb is not None (checked at the start of predict, but good practice)
            if self.mlb is None:
                 logging.error(f"MultiLabelBinarizer is None. Cannot perform inverse transform for {image_path}.")
                 return None

            binary_prediction: np.ndarray

            # Ensure predictions shape is compatible (must be 2D: (n_samples, n_classes))
            # Since we process one image at a time, expected shape is (1, n_classes)
            expected_shape: Tuple[int, int] = (1, len(self.mlb.classes_))

            if predictions.ndim == 1 and predictions.shape[0] == len(self.mlb.classes_):
                 binary_prediction = predictions.reshape(expected_shape)
                 logging.info(f"Reshaped 1D prediction to 2D ({expected_shape}) for inverse transform.")
            elif predictions.ndim == 2 and predictions.shape == expected_shape:
                 binary_prediction = predictions
                 logging.info(f"Prediction already in correct 2D shape ({expected_shape}) for inverse transform.")
            else:
                 logging.error(f"Cannot inverse transform prediction shape {predictions.shape} with MLB classes {len(self.mlb.classes_)} for {image_path}. Expected shape: {expected_shape}")
                 return None


            predicted_labels_tuple_list: List[Tuple[str, ...]] = self.mlb.inverse_transform(binary_prediction)
            logging.info(f"Prediction processed for {image_path}. Predicted labels (raw tuple list): {predicted_labels_tuple_list}")

            # inverse_transform returns a list of tuples, even for a single sample.
            # We expect a single prediction here, so we take the first tuple.
            if predicted_labels_tuple_list and len(predicted_labels_tuple_list) > 0:
                final_labels: List[str] = list(predicted_labels_tuple_list[0])
                logging.info(f"Final predicted labels for {image_path}: {final_labels}")
                return final_labels
            else:
                 logging.warning(f"MLB inverse_transform returned an empty list for {image_path}. No labels predicted.")
                 return []


        except Exception as e:
            logging.error(f"An error occurred during inverse transform for {image_path}: {e}", exc_info=True)
            return None

# LayoutLM

In [ ]:

from PIL import Image, ImageDraw, ImageFont
from transformers import LayoutLMv2ForSequenceClassification
from PIL import Image
import numpy as np
import torch
from typing import Optional, List, Dict, Any
import logging
from pathlib import Path
from transformers import LayoutLMv2ForSequenceClassification, LayoutLMv2Processor, LayoutLMv2FeatureExtractor, LayoutLMv2Tokenizer
import os
from dotenv import load_dotenv

try:
    load_model = LayoutLMv2ForSequenceClassification.from_pretrained("/kaggle/working/model.pth")
    model.to(device)
except Exception as e:
    raise e

def layout_model_prediction(img_path):



    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    id2label={0:'invoice',1: 'form', 2:'note', 3:'advertisement',4: 'email'}
    image = Image.open(img_path)
    image = image.convert("RGB")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # prepare image for the model
    encoded_inputs = processor(
    image,
    return_tensors="pt",
    truncation=True,
    padding="max_length",
    max_length=512)

    # make sure all keys of encoded_inputs are on the same device as the model
    for k,v in encoded_inputs.items():
      encoded_inputs[k] = v.to(model.device)

    load_model.to(device)
    # forward pass
    outputs = load_model(**encoded_inputs)
    logits = outputs.logits

    predicted_class_idx = logits.argmax(-1).item()
    print("Predicted class:", id2label[predicted_class_idx])

    return id2label[predicted_class_idx]

### Production grade

In [9]:
from PIL import Image
import numpy as np
import torch
from typing import Optional, List, Dict, Any
import logging
from pathlib import Path
from transformers import LayoutLMv2ForSequenceClassification, LayoutLMv2Processor, LayoutLMv2FeatureExtractor, LayoutLMv2Tokenizer
import os
from dotenv import load_dotenv

# Configure logging (if not already configured)
# Check if handlers exist to avoid adding multiple handlers in interactive environments
if not logging.getLogger('').handlers:
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info("Logging configured with basicConfig.")
else:
    logging.info("Logging already configured.")

# Load environment variables from .env file
# Set override=True to allow overriding existing environment variables
load_dotenv(override=True)

class LayoutLMDocumentClassifier:
    """
    A class for classifying documents using a LayoutLMv2 model.

    This class encapsulates the loading of the LayoutLMv2 model and its associated
    processor, handles image preprocessing, and performs document classification
    predictions. The model path is loaded from environment variables, promoting
    flexible configuration. It includes robust error handling, logging, and
    type hinting for production readiness.
    """

    def __init__(self,model_path_str) -> None:
        """
        Initializes the LayoutLMDocumentClassifier by loading the model and processor.

        The model and processor are loaded from the path specified in the
        environment variable 'LAYOUTLM_MODEL_PATH'. This method also sets up
        the device for inference (GPU if available, otherwise CPU) and defines
         the mapping from model output indices to class labels.

        Includes robust error handling and logging for initialization and artifact loading.

        Raises:
            ValueError: If the 'LAYOUTLM_MODEL_PATH' environment variable is not set.
            FileNotFoundError: If the model path specified in the environment variable
                               does not exist or a required artifact file is not found
                               during the artifact loading process.
            Exception: If any other unexpected error occurs during the loading
                       of the model or processor.
        """
        logging.info("Initializing LayoutLMDocumentClassifier.")
        self.model_path_str: Optional[str]=model_path_str
        self.model: Optional[LayoutLMv2ForSequenceClassification] = None
        self.processor: Optional[LayoutLMv2Processor] = None
        self.device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        logging.info(f"Using device: {self.device}")
        # Define id2label mapping as a class attribute
        # This mapping should align with the model's output classes.
        self.id2label: Dict[int, str] = {0:'invoice', 1: 'form', 2:'note', 3:'advertisement', 4: 'email'}
        logging.info(f"Defined id2label mapping: {self.id2label}")

        # Load model path from environment variable
        model_path_str: Optional[str] = self.model_path_str
        logging.info(f"Attempting to retrieve LAYOUTLM_MODEL_PATH from environment variables.")
        if not model_path_str:
            logging.critical("Critical Error: 'LAYOUTLM_MODEL_PATH' environment variable is not set.")
            raise ValueError("LAYOUTLM_MODEL_PATH environment variable is not set.")

        model_path: Path = Path(model_path_str)
        logging.info(f"Retrieved model path: {model_path}")
        if not model_path.exists():
             logging.critical(f"Critical Error: Model path from environment variable does not exist: {model_path}")
             raise FileNotFoundError(f"Model path not found: {model_path}")
        logging.info(f"Model path {model_path} exists.")


        try:
            logging.info("Calling _load_artifacts to load model and processor.")
            self._load_artifacts(model_path)
            if self.model is not None and self.processor is not None:
                logging.info("LayoutLMDocumentClassifier initialized successfully.")
            else:
                # This case should ideally be caught and re-raised in _load_artifacts
                logging.critical("LayoutLMDocumentClassifier failed to fully initialize due to artifact loading errors in _load_artifacts.")
                # _load_artifacts already raises on critical failure, no need to raise again
        except Exception as e:
            # Catch and log any exception that wasn't handled and re-raised in _load_artifacts
            logging.critical(f"An unhandled exception occurred during LayoutLMDocumentClassifier initialization: {e}", exc_info=True)
            raise # Re-raise the exception after logging
        logging.info("Initialization process completed.")


    def _load_artifacts(self, model_path: Path) -> None:
        """
        Loads the LayoutLMv2 model and processor from the specified path.

        This is an internal helper method called during initialization. It handles
        the loading of both the `LayoutLMv2ForSequenceClassification` model and
        its corresponding `LayoutLMv2Processor` with error handling and logging.

        Args:
            model_path: Path to the LayoutLMv2 model directory or file. This path
                        is expected to contain both the model weights and the
                        processor configuration/files.

        Raises:
            FileNotFoundError: If the `model_path` or any required processor/model
                               file within that path is not found.
            Exception: If any other unexpected error occurs during loading
                       from the specified path (e.g., corrupt files, compatibility issues).
        """
        logging.info(f"Starting artifact loading from {model_path} for LayoutLMv2.")
        processor_loaded: bool = False
        model_loaded: bool = False

        # Load Processor
        try:
            logging.info(f"Attempting to load LayoutLMv2 processor from {model_path}")
            # Load feature extractor and tokenizer separately to create the processor
            feature_extractor = LayoutLMv2FeatureExtractor()
            tokenizer = LayoutLMv2Tokenizer.from_pretrained("microsoft/layoutlmv2-base-uncased")
            self.processor = LayoutLMv2Processor(feature_extractor, tokenizer)
            logging.info("LayoutLMv2 processor loaded successfully.")
            processor_loaded = True
        except Exception as e:
            logging.critical(f"Critical Error: An unexpected error occurred while loading the LayoutLMv2 processor from {model_path}: {e}", exc_info=True)
            raise # Re-raise to indicate a critical initialization failure

        # Load Model
        try:
            logging.info(f"Attempting to load LayoutLMv2 model from {model_path}")
            self.model = LayoutLMv2ForSequenceClassification.from_pretrained(model_path)
            self.model.to(self.device) # Ensure model is on the correct device
            logging.info(f"LayoutLMv2 model loaded successfully and moved to {self.device}.")
            model_loaded = True
        except FileNotFoundError:
            logging.critical(f"Critical Error: LayoutLMv2 model file not found at {model_path}", exc_info=True)
            raise # Re-raise to indicate a critical initialization failure
        except Exception as e:
            logging.critical(f"Critical Error: An unexpected error occurred while loading the LayoutLMv2 model from {model_path}: {e}", exc_info=True)
            raise # Re-raise to indicate a critical initialization failure

        # Conditional logging based on loading success
        if model_loaded and processor_loaded:
             logging.info("All required LayoutLMv2 artifacts loaded successfully from _load_artifacts.")
        elif model_loaded and not processor_loaded:
             logging.error("LayoutLMv2 model loaded successfully, but processor loading failed in _load_artifacts.")
        elif not model_loaded and processor_loaded:
             logging.error("LayoutLMv2 processor loaded successfully, but model loading failed in _load_artifacts.")
        else:
            logging.error("Both LayoutLMv2 model and processor failed to load during _load_artifacts.")
        logging.info("Artifact loading process completed.")


    def _prepare_inputs(self, image_path: Path) -> Optional[Dict[str, torch.Tensor]]:
        """
        Loads and preprocesses an image to prepare inputs for the LayoutLMv2 model.

        This method handles loading the image from a file path, converting it to RGB,
        and using the loaded LayoutLMv2Processor to create the necessary input tensors
        (pixel values, input IDs, attention masks, bounding boxes). The tensors are
        then moved to the appropriate device for inference.

        Includes robust error handling and logging for each step.

        Args:
            image_path: Path to the image file (e.g., PNG, JPG) to be processed.

        Returns:
            A dictionary containing the prepared input tensors (e.g., 'pixel_values',
            'input_ids', 'attention_mask', 'bbox') as PyTorch tensors, if image
            loading and preprocessing are successful. Returns `None` if any
            step fails (e.g., file not found, image corruption, processor error).
        """
        logging.info(f"Starting image loading and preprocessing for {image_path}.")
        image: Optional[Image.Image] = None

        # Load image
        try:
            logging.info(f"Attempting to load image from {image_path}")
            image = Image.open(image_path)
            logging.info(f"Image loaded successfully from {image_path}.")
        except FileNotFoundError:
            logging.error(f"Error: Image file not found at {image_path}", exc_info=True)
            return None
        except Exception as e:
            logging.error(f"An unexpected error occurred while loading image {image_path}: {e}", exc_info=True)
            return None

        # Convert image to RGB
        try:
            logging.info(f"Attempting to convert image to RGB for {image_path}.")
            if image is None:
                 logging.error(f"Image is None after loading for {image_path}. Cannot convert to RGB.")
                 return None
            if image.mode != "RGB":
                image = image.convert("RGB")
                logging.info(f"Image converted to RGB successfully for {image_path}.")
            else:
                 logging.info(f"Image is already in RGB format for {image_path}.")

        except Exception as e:
            logging.error(f"An error occurred while converting image {image_path} to RGB: {e}", exc_info=True)
            return None


        # Prepare inputs using the processor
        if self.processor is None:
            logging.error("LayoutLMv2 processor is not loaded. Cannot prepare inputs.")
            return None

        encoded_inputs: Optional[Dict[str, torch.Tensor]] = None
        try:
            logging.info(f"Attempting to prepare inputs using processor for {image_path}.")
            # The processor expects a PIL Image or a list of PIL Images
            if image is None:
                 logging.error(f"Image is None before preprocessing for {image_path}. Cannot prepare inputs.")
                 return None

            encoded_inputs = self.processor(
                images=image,
                return_tensors="pt",
                truncation=True,
                padding="max_length",
                max_length=512
            )
            logging.info(f"Inputs prepared successfully for {image_path}.")
        except Exception as e:
            logging.error(f"An error occurred during input preparation for {image_path}: {e}", exc_info=True)
            return None

        # Move inputs to the device
        if encoded_inputs is not None:
            try:
                logging.info(f"Attempting to move inputs to device ({self.device}) for {image_path}.")
                for k, v in encoded_inputs.items():
                    if isinstance(v, torch.Tensor):
                        encoded_inputs[k] = v.to(self.device)
                logging.info(f"Inputs moved to device ({self.device}) successfully for {image_path}.")
            except Exception as e:
                logging.error(f"An error occurred while moving inputs to device for {image_path}: {e}", exc_info=True)
                return None
        else:
             logging.error(f"Encoded inputs are None after processing for {image_path}. Cannot move to device.")
             return None


        logging.info(f"Image loading and preprocessing completed successfully for {image_path}.")
        return encoded_inputs


    def predict(self, image_path: Path) -> Optional[str]:
        """
        Predicts the class label for a given image using the loaded LayoutLMv2 model.

        This is the main prediction method. It orchestrates the process by first
        preparing the image inputs using `_prepare_inputs`, performing inference
        with the LayoutLMv2 model, determining the predicted class index from the
        model's output logits, and finally mapping this index to a human-readable
        class label using the `id2label` mapping.

        Includes robust error handling and logging throughout the prediction pipeline.

        Args:
            image_path: Path to the image file to classify.

        Returns:
            The predicted class label as a string if the entire prediction process
            is successful. Returns `None` if any critical step fails (e.g.,
            image loading/preprocessing, model inference, or if the predicted
            index is not found in the `id2label` mapping).
        """
        logging.info(f"Starting prediction process for image: {image_path}.")

        # Prepare inputs
        logging.info(f"Calling _prepare_inputs for {image_path}.")
        encoded_inputs: Optional[Dict[str, torch.Tensor]] = self._prepare_inputs(image_path)
        if encoded_inputs is None:
            logging.error(f"Input preparation failed for {image_path}. Cannot perform prediction.")
            logging.info(f"Prediction process failed for {image_path}.")
            return None
        logging.info(f"Input preparation successful for {image_path}.")


        # Check if model is loaded
        if self.model is None:
            logging.error("LayoutLMv2 model is not loaded. Cannot perform prediction.")
            logging.info(f"Prediction process failed for {image_path}.")
            return None
        logging.info("LayoutLMv2 model is loaded. Proceeding with inference.")

        predicted_label: Optional[str] = None

        try:
            logging.info(f"Performing model inference for {image_path}.")
            self.model.eval() # Set model to evaluation mode
            with torch.no_grad():
                outputs: Any = self.model(**encoded_inputs)
                logits: torch.Tensor = outputs.logits

            # Determine predicted class index
            # Ensure logits is a tensor before calling argmax
            if not isinstance(logits, torch.Tensor):
                 logging.error(f"Model output 'logits' is not a torch.Tensor for {image_path}. Cannot determine predicted index.")
                 logging.info(f"Prediction process failed for {image_path} due to invalid model output.")
                 return None

            predicted_class_idx: int = logits.argmax(-1).item()
            logging.info(f"Model inference completed for {image_path}. Predicted index: {predicted_class_idx}.")

            # Map index to label
            logging.info(f"Attempting to map predicted index {predicted_class_idx} to label.")
            if predicted_class_idx in self.id2label:
                predicted_label = self.id2label[predicted_class_idx]
                logging.info(f"Mapped predicted index {predicted_class_idx} to label: {predicted_label}.")
            else:
                logging.error(f"Predicted index {predicted_class_idx} not found in id2label mapping for {image_path}.")
                logging.info(f"Prediction process failed for {image_path} due to unknown predicted index.")
                return None # Return None if index is not in mapping

        except Exception as e:
            logging.error(f"An error occurred during model inference or label mapping for {image_path}: {e}", exc_info=True)
            logging.info(f"Prediction process failed for {image_path} due to inference/mapping error.")
            return None

        logging.info(f"Prediction process completed successfully for {image_path}. Predicted label: {predicted_label}.")
        return predicted_label